# QE input script generator
This will automate the process of writing input scripts for Quantum Espresso to do the structural optimization of functionalized mofs using the pw.x (plane wave) package and pseudopotential files. 

**NOTE:** many default arguements will be preset.

In [2]:
using PorousMaterials
using Printf

In [18]:
@eval PorousMaterials PATH_TO_CRYSTALS = joinpath(pwd(), "mof_construction", "NiPyC2_relax") 
# PATH_TO_PSEUDOPOTENTIALS = joinpath(pwd(), "data", "pseudo")

"/home/cokes/DTRA/mof_construction/NiPyC2_relax"

In [19]:
# The name of the fragment
# file must be a .xyz file located in ./fragments/
# The atom species that is bonded to the :C_aro_R atom on the ring segment of the Fragment
fragment_list = ["Br", "CH2-CH2-CH3", "CH2-CH3", "CH3", 
                "Cl", "C-N", "F", "NH2", "N-NH", 
                "O-CH2-CH2-CH3", "O-CH2-CH3", "O-CH3", "OH"]

# The type of Arene Substitution ("ortho", "meta", "all") 
# TODO: "all" not yet implimented
substitution_types = ["ortho", "meta"]

2-element Array{String,1}:
 "ortho"
 "meta" 

In [20]:
base_name = "NiPyC2_relax"
xtals = Dict{String, Crystal}()
for frag in fragment_list
    for sub_typ in substitution_types
        xtal_name = base_name * "_" * sub_typ * "_functionalized_" * frag
        xtals[xtal_name] = Crystal(xtal_name * ".cif")
    end
end

SystemError: SystemError: opening file "/home/cokes/DTRA/mof_construction/NiPyC2_relax/NiPyC2_relax_ortho_functionalized_Br.cif": No such file or directory

In [6]:
# populate dictionary with names of pseudopotential files
# note: may need to ignore case sensitivity?
# ex: atomic_psp[:Ni] = ["ni_pbesol_v1.4.uspp.F.UPF"]

ps_lib = Dict{Symbol, String}()
ps_lib

Dict{Symbol,String} with 0 entries

In [7]:
# create a dictionary consisting of input arguements
# needed for the script and set default values
# then populate with values specific to each material

In [29]:
# import the functionalized mofs 
crystal = Crystal("NiPyC2_relax_meta_functonalized_NH2.cif")
strip_numbers_from_atom_labels!(crystal)

In [30]:
filename = "pw." * crystal.name * ".in"

"pw.NiPyC2_relax_meta_functonalized_NH2.cif.in"

In [32]:
qe_input_file = open(filename, "w")

# write &Control
@printf(qe_input_file,
    """
    &CONTROL
    calculation = 'relax'
    prefix = '%s'
    pseudo_dir = 'pseudo/'
    ! etot_conv_thr=1.0d-6,
    ! 
    ! wf_collect=.false.
    """,
    crystal.name
)

# write ATOMIC_POSITIONS
#symbol #x #y #z
ni_count = 1
for a = 1:crystal.atoms.n
    if crystal.atoms.species[a] == :Ni
        n_count += 1
    end
    @printf("%s    %f  %f  %f\n",
        crystal.atoms.species[a], 
        crystal.atoms.coords.xf[1, a], 
        crystal.atoms.coords.xf[2, a], 
        crystal.atoms.coords.xf[3, a]
    )
end

close(qe_input_file)

run(`cat $filename`)

UndefVarError: UndefVarError: n_count not defined

In [8]:
# use %s for strings, 

@printf(f,
    """
    &CONTROL
    calculation = 'relax'
    prefix = '%s'
    pseudo_dir = 'pseudo/'
    ! etot_conv_thr=1.0d-6,
    ! 
    ! wf_collect=.false.
    /
    &SYSTEM
    ibrav = 0,
    nat = %.0f,
    ntyp = %.0f,
    tot_charge = %f,
    ecutwfc = %f,
    vdw_corr='grimme-d2'
    /
    &ELECTRONS
    diagonalization='david',
    /
    &IONS
    ion_dynamics='bfgs',
    /
    &CELL
    cell_dynamics='bfgs'
    cell_dofree='xyz'
    /
    ATOMIC_SPECIES
    #symbol #atomic_mass #pseudo
    CELL_PARAMETERS (angstrom)
    # matrix
    ATOMIC_POSITIONS
    #symbol #x #y #z
    K_POINTS {gamma}
    """,
    
    args)

LoadError: ArgumentError: @printf: wrong number of arguments (1) should be (5)

In [9]:
close(f)

UndefVarError: UndefVarError: f not defined

In [10]:
# how do I write the cell parameters and
# atomic positions as a formatted string?